# Entropy Monkey - flip the coin strategy

An example how to randomly go all-in to a new token every day.

This is not a real strategy, but an example of different facilities available for the trade: how to construct a trading pair universe, how to check price, how to buy, how to close position

* Tokens need to have have USD 500k+ all-time cumulative volume before it can be chosen

* This is a simplified example strategy that ignores available liquidity and loss of trade balance due to slippage, currency conversions, etc.

* This is not a Monte-Carlo simulation. The outcome of the strategy depends purely on which seed number you choose to initialise the random number generator. Basically, your skill is 100% of luck.

## Creating trading universe

First let's import libraries and initialise our dataset client.

In [36]:
from random import Random

import pandas as pd

from capitalgram.chain import ChainId
from capitalgram.pair import PairUniverse, PandasPairUniverse

try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram

capitalgram = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/moo/.capitalgram


Let's create a pair universe for Sushi. [See full example](https://docs.capitalgram.com/examples/pairs.html).
We will create a dataset of 4h candles that trade on Sushiswap on Ethereum.

In [37]:
# Decompress the pair dataset to Python map
columnar_pair_table = capitalgram.fetch_pair_universe()

# Exchange map data is so small it does not need any decompression
exchange_universe = capitalgram.fetch_exchange_universe()

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
# and also have (had) some meaningful volume in some point of their life
sushi_swap = exchange_universe.get_by_name_and_chain(ChainId.ethereum, "sushiswap")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[
    (all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id) &  # Trades on Sushi
    (all_pairs_dataframe['buy_volume_all_time'] > 500_000)  # 500k min buys
]

# Create a Python set of pair ids
wanted_pair_ids = sushi_pairs["pair_id"]

# Make the trading pair data easily accessible
pair_universe = PandasPairUniverse(sushi_pairs)

print(f"Sushiswap on Ethereum has {len(pair_universe.get_all_pair_ids())} legit trading pairs")

Sushiswap on Ethereum has 311 legit trading pairs


Get daily candles and filter them against our wanted pair set.

In [38]:
import datetime

from capitalgram.candle import CandleBucket, GroupedCandleUniverse
from capitalgram.pair import PandasPairUniverse
from capitalgram.frameworks.backtrader import prepare_candles_for_backtrader

# Get daily candles as Pandas DataFrame
all_candles = capitalgram.fetch_all_candles(CandleBucket.h24).to_pandas()
sushi_candles: pd.DataFrame = all_candles.loc[all_candles["pair_id"].isin(wanted_pair_ids)]

sushi_candles = prepare_candles_for_backtrader(sushi_candles)

# We limit candles to a specific date range to make this notebook deterministic
start = datetime.datetime(2020, 10, 1)
end = datetime.datetime(2021, 6, 1)

sushi_candles = sushi_candles[(sushi_candles.index >= start) & (sushi_candles.index <= end)]

# Group candles by the trading pair ticker
sushi_tickers = GroupedCandleUniverse(sushi_candles)

print(f"Candles to feed to the strategy, all pairs is {len(sushi_candles)}")


Candles to feed to the strategy, all pairs is 32457


## Creating coin flip backtrader strategy

[See the Backtrader quickstart tutorial](https://www.backtrader.com/docu/quickstart/quickstart/).

### Cumulative volume indicator

We create a [Backtrader indicator based on PeriodN indicator](https://github.com/mementum/backtrader/blob/0fa63ef4a35dc53cc7320813f8b15480c8f85517/backtrader/indicators/basicops.py) how much cumulative trade volume the trading pair has seen in the past. We use this indicator to filter out trading pairs that seem to be dead, as no trading happening, and thus not subject to our random entry.

More information about [Backtrader custom indicators](https://www.backtrader.com/docu/inddev/).

In [39]:
import math

import backtrader as bt
from backtrader.indicators import PeriodN


class PastTradeVolumeIndicator(PeriodN):
    """Indicates whether the trading pair has reached certain volume for the last N days.

    Based on indicator base class that takes period (days) as an input.
    """

    lines = ('cum_volume',)

    def next(self):
        # This indicator is feed with volume line.
        # We simply take the sum of the daily volumes based on the period (number of days)
        datasum = math.fsum(self.data.get(size=self.p.period))
        self.lines.cum_volume[0] = datasum


### Coinflip strategy core

By using the indicator from the above here is our strategy.

In [40]:
from capitalgram.frameworks.backtrader import CapitalgramFeed

class EntropyMonkey(bt.Strategy):
    """A strategy that picks a new token to go all-in every day."""

    def __init__(self, pair_universe: PandasPairUniverse, seed: int):
        #: Allows us to print human-readable pair information
        self.pair_universe = pair_universe

        #: Initialize (somewhat) determininistic random number generator
        self.dice = Random(seed)

        #: We operate on daily candles.
        #: At each tick, we process to the next candle
        self.day = 0

        #: Cumulative volume indicator for each of the data feed
        self.indicators = {}
        pair: CapitalgramFeed
        for pair in self.datas:
            self.indicators[pair] = PastTradeVolumeIndicator(pair.lines.volume)

        # How much USD volume token needs to have in order to be eligible for a pick
        self.cumulative_volume_threshold = 500_000

        # How many times we try to pick a token pair to buy
        # before giving up (at early days there might not be enough volume)
        self.pick_attempts = 100

        # If our balance goes below this considering giving up
        self.cash_balance_death_threshold = 100

    def next(self):
        """Tick the strategy.

        Because we are using daily candles, tick will run once per each day.
        """

        # Advance to the next day
        self.day += 1

        # Pick a new token to buy
        for i in range(self.pick_attempts):
            random_pair: CapitalgramFeed = self.dice.choice(self.datas)
            pair_info = random_pair.pair_info
            cum_volume_indicator = self.indicators[random_pair]
            volume = cum_volume_indicator.lines.cum_volume[0]
            if volume > self.cumulative_volume_threshold:
                break
        else:
            print(f"On day #{self.day} did not find any token to buy")
            return

        # Sell any existing token we have
        for ticker in self.datas:
             if self.getposition(ticker).size > 0:
                print(f"On day #{self.day}, selling existing position of {pair_info.base_token_symbol} - {pair_info.quote_token_symbol}")
                self.close(ticker)

        # Buy in with all money we have.
        # We are not really worried about order size quantisation in crypto.
        cash = self.broker.get_cash()

        if cash < self.cash_balance_death_threshold:
            # We are busted
            return

        # Buy using the daily candle closing price as the rate
        price = random_pair.close[0]
        assert price > 0
        size = cash / price

        # Sell the existing position
        print(f"On day #{self.day} we are buying {pair_info.base_token_symbol} - {pair_info.quote_token_symbol} that has all-time vol of {volume}. Buy in at the close price of {price} {pair_info.base_token_symbol}, cash at hand {cash} USD")

        self.buy(random_pair, size=size, exectype=bt.Order.Market)


## Feed the strategy

Feed in Sushiswap data to the backtrader strategy

In [41]:
from capitalgram.frameworks.backtrader import add_dataframes_as_feeds

# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

# Add a strategy
cerebro.addstrategy(EntropyMonkey, pair_universe=pair_universe, seed=0x1000)

# Pass all Sushi pairs to the data fees to the strategy
# noinspection JupyterKernel
feeds = [df for pair_id, df in sushi_tickers.get_all_pairs()]
add_dataframes_as_feeds(
    cerebro,
    pair_universe,
    feeds,
    start,
    end,
    CandleBucket.h24)

## Running the strategy

We are adding some observers that tell us how well the strategy performs.

In [42]:
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)

# Run over everything
print('Starting portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Ending portfolio Value: %.2f' % cerebro.broker.getvalue())

strategy: EntropyMonkey = results[0]

# The number of ticks the strategy performed
assert strategy.day == 244, f"Simulated trading for {strategy.day} days"

Starting portfolio Value: 10000.00
On day #1 we are buying COMP - WETH that has all-time vol of 580770.4375. Buy in at the close price of 127.49980926513672 COMP, cash at hand 10000.0 USD
On day #2 we are buying SNX - WETH that has all-time vol of 1738957.25. Buy in at the close price of 4.367995262145996 SNX, cash at hand 10000.0 USD
On day #3, selling existing position of <Pair SNX - WETH (0xa1d7b2d891e3a1f9ef4bbc5be20630c2feb1c470) at exchange #22 on Ethereum>
On day #4 we are buying WETH - USDT that has all-time vol of 3303272.25. Buy in at the close price of 352.5541076660156 WETH, cash at hand 10073.6532992735 USD
On day #5 we are buying SUSHI - WETH that has all-time vol of 4828287.0. Buy in at the close price of 0.8386799693107605 SUSHI, cash at hand 10073.6532992735 USD
On day #6 we are buying SNX - WETH that has all-time vol of 1774679.875. Buy in at the close price of 3.632591724395752 SNX, cash at hand 10073.6532992735 USD
On day #7 we are buying BAND - WETH that has all-ti

## Analysing the portfolio results

In [43]:
print(f"Backtest range {start} - {end}")
print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
print(f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

Final Portfolio Value: 143463.43
Sharpe: 0.976
Norm. Annual Return: 1465.55%
Max Drawdown: 95.00%


## Plotting the portfolio result

In [44]:
pass

## Notes

[Adding custom DataFrames to Backtrader](https://community.backtrader.com/topic/1828/how-to-feed-a-custom-pandas-dataframe-in-backtrader)
